## Reverend Bayes' legendary coin of wisdom
### A parable

An eminent Professor of History at your university has recently discovered a relic that was once owned by the venerable Reverend Bayes. The item is the legendary "coin of wisdom", said to impart those who possess it with powerful knowledge of statistics. Legend has it that despite appearing to be a fair coin, the coin is biased, with an unequal probability of producing heads or tails. As the Professor of History could barely muster an addition if his life depended on it, he has reached out for collaborative assistance from our group to either confirm or deny the legend.

The coin flipping problem is of course a well-known problem from statistics, and when probabilities of heads or tails (or "probability of success" if we define success to be either heads or tails) are unequal the process can be modeled using the binomial distribution.

You have been tasked with estimating the probability that a flip of the coin produces tails (lets define tails as "success"). To do so, you collect a dataset of three sets of coin flips:
1. Your professor flips the coin five times, producing 1 tails and 4 heads, before realising he is late to an important meeting and handing the coin to a postdoc in the group.
2. The postdoc in the group flips the coin ten times, producing 5 tails and 5 heads, before realising it's time to collect the child from daycare and giving you the coin.
3. You flip the coin 100 times, producing 66 tails and 34 heads, before noticing how hungry you are and setting out to have dinner.

Because each person may have slight different biases in how they flip coins you decide not to pool all of the results, but instead to treat them as independent estimates under the binomial distribution. And for no particular reason, you decide to take the maximum likelihood computed under the binomial at fixed probabilities of tails/success rather than using analytical estimation or some kind of gradient optimisation.

Therefore, you calculate the likelihood of the three observations above when probability of tails ranges from 0.05, 0.1 ... 0.95 in increments of 0.05.

What is the maximum likelihood value of the probability of tails, p_t?

The likelihood that an arbitrary value p_t produced an observed set of coin flips can be calculated using the probability mass function of the binomial.


In [1]:
import scipy.stats as stats
# compute probability mass at each level of fairness for 
# the professor's coin flips
print("Professor:")
for i in range(1,20):
  print("\tp_t = "+str(i/20)+", likelihood="+str(stats.binom.pmf(1, 5, i/20)))

# now do the same for the postdoc's coin flips
print("Postdoc:")
for i in range(1,20):
  print("\tp_t = "+str(i/20)+", likelihood="+str(stats.binom.pmf(5, 10, i/20)))

# and for the student's coin flips
print("Student:")
for i in range(1,20):
  print("\tp_t = "+str(i/20)+", likelihood="+str(stats.binom.pmf(66, 100, i/20)))


Professor:
	p_t = 0.05, likelihood=0.20362656250000014
	p_t = 0.1, likelihood=0.32804999999999995
	p_t = 0.15, likelihood=0.39150468749999984
	p_t = 0.2, likelihood=0.4095999999999999
	p_t = 0.25, likelihood=0.39550781250000006
	p_t = 0.3, likelihood=0.3601499999999999
	p_t = 0.35, likelihood=0.31238593750000004
	p_t = 0.4, likelihood=0.2592000000000001
	p_t = 0.45, likelihood=0.20588906250000005
	p_t = 0.5, likelihood=0.15624999999999997
	p_t = 0.55, likelihood=0.11276718749999998
	p_t = 0.6, likelihood=0.0768
	p_t = 0.65, likelihood=0.048770312499999996
	p_t = 0.7, likelihood=0.02834999999999999
	p_t = 0.75, likelihood=0.014648437499999998
	p_t = 0.8, likelihood=0.006399999999999999
	p_t = 0.85, likelihood=0.0021515624999999994
	p_t = 0.9, likelihood=0.0004499999999999999
	p_t = 0.95, likelihood=2.968750000000011e-05
Postdoc:
	p_t = 0.05, likelihood=6.093524882812493e-05
	p_t = 0.1, likelihood=0.0014880347999999995
	p_t = 0.15, likelihood=0.008490855786328114
	p_t = 0.2, likelihood=0

## combining information from the separate trials

We have now obtained the likelihoods of different p_t for each of the three independent coin flip trials. How do we combine them? One approach would be to sum the (log) likelihoods across trials and take the maximum. Does it produce the correct result? Which trial contributes the most to the estimate of p_t?



## A further correspondence from the historian

The historian has come into contact with a colleague in Europe who is in possession of a letter written by the reverend that describes the manufacture of the coin. In it, the reverend indicates the shape and weight of the coin were designed to produce tails 60% of the time. How does that compare to your estimate? How does it compare to the student's estimate?

In [2]:
# scale factor
import numpy as np
prof_lk = np.log([stats.binom.pmf(1, 5, i/20) for i in range(1,20)])
postdoc_lk = np.log([stats.binom.pmf(5, 10, i/20) for i in range(1,20)])
student_lk = np.log([stats.binom.pmf(66, 100, i/20) for i in range(1,20)])
avg_prof_lk = np.average(prof_lk)
avg_postdoc_lk = np.average(postdoc_lk)
avg_student_lk = np.average(student_lk)
print(avg_prof_lk,avg_postdoc_lk,avg_student_lk)
print(avg_prof_lk/avg_postdoc_lk, avg_prof_lk/avg_student_lk) # scaling factor

-3.016622353441251 -3.7226914442392793 -30.894898798704475
0.810333705768002 0.09764143825477582


In [3]:
0.010422094818650304*16.728282848474226

0.17434374999999996

In [4]:
postdoc_lk_scaled = np.array(postdoc_lk) * (avg_prof_lk/avg_postdoc_lk)
student_lk_scaled = np.array(student_lk) * (avg_prof_lk/avg_student_lk)

In [5]:
import pandas as pd
probabilities = [i/20 for i in range(1,20)]
prof_vals = pd.Series(prof_lk,probabilities)
postdoc_vals = pd.Series(postdoc_lk_scaled,probabilities)
student_vals = pd.Series(student_lk_scaled,probabilities)
postdoc_vals

0.05   -7.864855
0.10   -5.275515
0.15   -3.864291
0.20   -2.944330
0.25   -2.301715
0.30   -1.842545
0.35   -1.518238
0.40   -1.301520
0.45   -1.176843
0.50   -1.136122
0.55   -1.176843
0.60   -1.301520
0.65   -1.518238
0.70   -1.842545
0.75   -2.301715
0.80   -2.944330
0.85   -3.864291
0.90   -5.275515
0.95   -7.864855
dtype: float64

In [6]:
collected_df = pd.concat([prof_vals,postdoc_vals,student_vals], axis=1)
collected_df

,0,1,2
0.05,-1.591468,-7.864855,-13.458505
0.10,-1.114589,-5.275515,-9.171125
0.15,-0.937758,-3.864291,-6.747927
0.20,-0.892574,-2.944330,-5.095270
0.25,-0.927585,-2.301715,-3.871513
0.30,-1.021235,-1.842545,-2.925615
0.35,-1.163516,-1.518238,-2.178241
0.40,-1.350155,-1.301520,-1.583447
0.45,-1.580418,-1.176843,-1.113274
0.50,-1.856298,-1.136122,-0.750708


In [7]:
collected_df_sum = collected_df.sum(axis=1)
collected_df_sum

0.05   -22.914827
0.10   -15.561229
0.15   -11.549976
0.20    -8.932174
0.25    -7.100813
0.30    -5.789395
0.35    -4.859995
0.40    -4.235122
0.45    -3.870535
0.50    -3.743128
0.55    -3.845547
0.60    -4.184631
0.65    -4.782909
0.70    -5.683884
0.75    -6.964007
0.80    -8.759544
0.85   -11.333973
0.90   -15.287617
0.95   -22.548168
dtype: float64